In [1]:
import keyring

# OpenAI API keu
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')

In [2]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'    
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()
        
        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
    else:
        print(f"Failed to fetch article at {article_url}")
        
except Exception as e:
    print(f"Error occurred while fetching article {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry) or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the fastest in the world once complete and 

In [3]:
from langchain.schema import (
    HumanMessage
)

# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change'
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather coditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==========
Title: {article_title}

{article_text}
==========

Please provide a summarized version of the article in a bulleted list format.
"""

# format prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [4]:
from langchain.chat_models import ChatOpenAI

# load model
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)

# generate summary
summary = chat(messages)
print(summary.content)

C:\Users\woojin\AppData\Local\Temp\ipykernel_34032\3864175484.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=OPENAI_API_KEY)
C:\Users\woojin\AppData\Local\Temp\ipykernel_34032\3864175484.py:7: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chat(messages)


- Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that aims to be the world's fastest.
- The RSC is being used for training large natural language processing (NLP) and computer vision models and is set to be fully built by mid-2022.
- Meta hopes that the RSC will enable the development of new AI systems for applications like real-time voice translations and AR games in the metaverse.
- The RSC is expected to be 20x faster than Meta's current clusters and significantly improve training times for large-scale NLP workflows.
- Meta designed the RSC with security and privacy controls to use real-world data from its production systems for research purposes.
- The RSC will help Meta advance research in tasks such as identifying harmful content on its platforms.
- The AI & Big Data Expo will feature industry leaders discussing AI and big data topics in upcoming events in Santa Clara, Amsterdam, and London.


## With OutputParser

In [5]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import field_validator
from pydantic import BaseModel, Field
from typing import List

# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")
    
    # validating whether the generated summary has at least three lines
    @field_validator('summary')
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines
    
# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

In [6]:
from langchain.prompts import PromptTemplate

# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==========
Title: {article_title}

{article_text}
==========

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# instantiate model class
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

chain = LLMChain(llm=model, prompt=prompt_template)

# run the LLMChain to get the AI-generated answer
output = chain.run({"article_title":article_title, "article_text":article_text})

# parse the output into the Pydantic model
parsed_output = parser.parse(output)
print(parsed_output)

C:\Users\woojin\AppData\Local\Temp\ipykernel_34032\3625487116.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=model, prompt=prompt_template)
C:\Users\woojin\AppData\Local\Temp\ipykernel_34032\3625487116.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chain.run({"article_title":article_title, "article_text":article_text})


title='Meta claims its new AI supercomputer will set records' summary=["Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world's fastest.", "RSC is still under construction but Meta's researchers have already started using it for training large NLP and computer vision models.", 'Once fully built in mid-2022, RSC is expected to be the fastest in the world and capable of training models with trillions of parameters.', 'Meta aims to use RSC to develop new AI systems for applications like real-time voice translations and AR games in the metaverse.', "RSC is projected to be 20x faster than Meta's current clusters, 9x faster at running NCCL, and 3x faster at training large-scale NLP workflows.", 'With RSC, models with billions of parameters can finish training in three weeks compared to nine weeks previously.', 'Meta designed RSC with security and privacy controls to use real-world data from its production systems 